In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib
import matplotlib.pyplot as plt
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180

import sys
sys.path.append('/home/cat/code/gerbil/utils/')

#
import os
import numpy as np

#
from visualize import Visualize
from track import Track


from tqdm import trange

#
import sleap


ModuleNotFoundError: No module named 'tkinter'

In [7]:

fname = "/media/cat/256GB/dan/testing_track_cleanup_code/cohort2_night_1000/videos/cohort2_night.1000.slp"
labels = sleap.load_file(fname)

labels.videos  # Use in ipython to print list of videos in the labels object
n_videos = len(labels.videos)

#
#huddle_location = np.array([150,150])
#min_dist = 100
huddle_block = np.array([200,
                         200])


#
n_all=0
for video_idx in trange(n_videos):
    #video_idx =   # Change this select the video of interest from the labels.video list
    video = labels.videos[video_idx]
    labeled_frames_from_video = labels.get(video)

    # Loop through all labeled frames (from a specific video)
    n_frames = 0
    for ctr1, lf in enumerate(labeled_frames_from_video):

        # Loop through all user instances in the current labeled frame
        for ctr2, inst in enumerate(lf.user_instances):

            #
            remove_inst = False
            points_array = inst.points_array  # Returns a (num_nodes, 2) np.recarray

            # remove point wise features
            if False:
                diff = points_array - huddle_location
                dists = np.linalg.norm(diff,axis=1)
            
            # this sometimes leaves chimera track in
            if False:
                median = np.nanmedian(points_array,axis=0)
                #median = np.nanmean(points_array,axis=0)
                #print ("median: ", median)
                dist = np.linalg.norm(median-huddle_location)
            
            # RADIUS EXCLUDE this checks for any features at all in the exclusion radius
            if False:
                idx = np.where(np.isnan(points_array.sum(1))==False)[0]
                points_array = points_array[idx]
                dist = np.linalg.norm(points_array-huddle_location, axis=1)
                dist = np.min(dist)
                
                # delete tracks 
                if dist<min_dist:
                    lf.instances.remove(inst)

            # SQUARE EXCLUDE this checks for any features at all in the exclusion radius
            if True:
                idx = np.where(np.isnan(points_array.sum(1))==False)[0]
                points_array = points_array[idx]
                
                # check the x and y locatoins
                idx1 = np.where(points_array[:,0]<huddle_block[0])[0]
                idx2 = np.where(points_array[:,1]<huddle_block[1])[0]
                
                # if any point is in both lists then it's in the huddle zone
                if set(idx1) & set(idx2):
                    lf.instances.remove(inst)
                    #
                #dist = np.linalg.norm(points_array-huddle_location, axis=1)
                #dist = np.min(dist)


            #print ('')

# Save the modified slp
new_filename = fname[:-4]+"_deleted.slp"  # Use a different path from the current slp to be safe
labels.save_file(labels, new_filename)
            

100%|██████████| 12/12 [00:01<00:00, 11.43it/s]


In [8]:
####################################################
############ VISUALIZE DELETED TRACKS ##############
####################################################

fname = "/media/cat/256GB/dan/testing_track_cleanup_code/cohort2_night_1000/videos/cohort2_night.1000_deleted.slp"
labels = sleap.load_file(fname)

labels.videos  # Use in ipython to print list of videos in the labels object
n_videos = len(labels.videos)

#
locs = []
for video_idx in trange(n_videos):
    #video_idx =   # Change this select the video of interest from the labels.video list
    video = labels.videos[video_idx]
    labeled_frames_from_video = labels.get(video)

    # Loop through all labeled frames (from a specific video)
    n_frames = 0
    for ctr1, lf in enumerate(labeled_frames_from_video):

        # Loop through all user instances in the current labeled frame
        for ctr2, inst in enumerate(lf.user_instances):

            #
            remove_inst = False
            points_array = inst.points_array  # Returns a (num_nodes, 2) np.recarray
            
            #
            #print (points_array[:,0])
            #print (points_array[:,1])
            idx1 = np.where(points_array[:,0]<200)[0]
            idx2 = np.where(points_array[:,1]<200)[0]
            
            inter = list(set(idx1) & set(idx2))
            locs.append(points_array)
            
            n_all+=1
            n_frames+=1
    print (n_frames)
    
print (n_all)
locs = np.vstack(locs)
print ("locs: ", locs.shape)

#
plt.figure()
plt.scatter(locs[:,0],
            locs[:,1])
plt.xlim(0,900)
plt.ylim(0,700)
plt.show()


 33%|███▎      | 4/12 [00:00<00:00, 25.63it/s]

127
205
173
359
287


 58%|█████▊    | 7/12 [00:00<00:00, 17.16it/s]

464
512
306


 92%|█████████▏| 11/12 [00:00<00:00, 15.17it/s]

590
354
377


100%|██████████| 12/12 [00:01<00:00, 11.81it/s]

504
4258
locs:  (25548, 2)
